In [155]:
import json
from openai import OpenAI
import pandas as pd
from IPython.display import Image, display
import hashlib
import json

client = OpenAI(api_key="sk-proj-sScFMZr4GK3XaQ5mcYFYT3BlbkFJ1orIjm1gKSNT0q30KBMZ")

In [156]:
resumes = pd.read_csv('data/trunc-resumes.csv')
postings = pd.read_csv('data/trunc-postings.csv')
resumes = resumes.head(10)
postings = postings.head(10)
display(postings)
display(resumes)

,description
0,Job descriptionA leading real estate firm in N...
1,"At Aspen Therapy and Wellness , we are committ..."
2,The National Exemplar is accepting application...
3,Senior Associate Attorney - Elder Law / Trusts...
4,Looking for HVAC service tech with experience ...
5,Job summary:The Economic Development & Plannin...
6,Company DescriptionRaw Cereal is a creative de...
7,Summary: Due to the pending retirement of our ...
8,"At Children’s, the region’s only full-service ..."
9,It is an exciting time to be a part of our chu...


,Resume,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR
5,HR GENERALIST Summary Dedic...,HR
6,HR MANAGER Summary HUMAN RES...,HR
7,HR MANAGER Professional Summa...,HR
8,HR SPECIALIST Summary Posses...,HR
9,HR CLERK Summary Translates ...,HR


In [157]:
system_prompt="""
Your goal is to use someone's resume to tell me if they are qualified for a job.
You will be provided with a resume and a job description. Your output will be a json object containing only one of five categorizations: "not qualified", "minimally qualified", "somewhat qualified", "qualified", "very qualified".
{
	categorization: string // Level of qualification
} 
"""

def Get_Qualifications(resume, job_description):
	user_prompt = f"Here is my resume:{resume}, and here is the job description: {job_description}"
	response = client.chat.completions.create(
		model="gpt-3.5-turbo",
		temperature=0.1,
		response_format={"type":"json_object"},
		messages=[
			{ "role": "system", "content":system_prompt },
			{ "role": "user", "content": user_prompt }
		]
	)
	return response.choices[0].message.content

In [158]:
for i in range(3):
	resume = resumes['Resume'][i]
	job_description = postings['description'][i]
	result = Get_Qualifications(resume, job_description)
	print(result)

{
    "categorization": "somewhat qualified"
}
{
    "categorization": "somewhat qualified"
}
{
    "categorization": "not qualified"
}


In [159]:
# Creating an array of json tasks
tasks = []
for job_description in postings['description']:
	for resume_text in resumes['Resume']:
		user_prompt = f"Here is my resume:{resume}, and here is the job description: {job_description}"
		key = hashlib.md5(f"{resume_text}{job_description}".encode()).hexdigest()

		task = {
        	"custom_id": f"{key}",
        	"method": "POST",
        	"url": "/v1/chat/completions",
        	"body": {
            	"model": "gpt-3.5-turbo",
            	"temperature": 0.1,
            	"response_format": { "type": "json_object" },
            	"messages": [
                	{ "role": "system", "content": system_prompt },
                	{ "role": "user", "content": user_prompt }
				]
			}
		}
		tasks.append(task)

In [160]:
# Creating the batch file
file_name = "data/batch_tasks_labels.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [161]:
# Uploading the Batch file to OpenAI
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)
print(batch_file)

FileObject(id='file-a4noWdptMuzcfKe8yKlvY7zy', bytes=1103550, created_at=1720413819, filename='batch_tasks_labels.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [162]:
# Creating the batch job:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [165]:
# Checking the batch status
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_PALdxQXBJjLFQnlR8hObeUzj', completion_window='24h', created_at=1720413819, endpoint='/v1/chat/completions', input_file_id='file-a4noWdptMuzcfKe8yKlvY7zy', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1720413856, error_file_id=None, errors=None, expired_at=None, expires_at=1720500219, failed_at=None, finalizing_at=1720413852, in_progress_at=1720413820, metadata=None, output_file_id='file-MDNaPIpcXWTAADuLUjinV3U1', request_counts=BatchRequestCounts(completed=100, failed=0, total=100))


In [166]:
# Retrieving results
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

result_file_name = "data/batch_job_results_labels.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)
    
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [167]:
# Create a cache from the results
cache = {}
for res in results:
	key = res['custom_id']
	task_id = res['custom_id']
	result_str = res['response']['body']['choices'][0]['message']['content']
	result_obj = json.loads(result_str)
	result = result_obj['categorization']
	if result in ("qualified", "very qualified"):
		cache[key] = 1
	else:
		cache[key] = 0
	print(f"{task_id},{result}")

with open('cache.json', 'w') as cache_file:
		json.dump(cache, cache_file)

0a5e37c7646026f3a77a258c30b37ee4,not qualified
b4c0fe30abe4a0e486417f6b162ca4c7,not qualified
f6b1143d6a4581950c3413f25592ec89,somewhat qualified
b3cfc85063de75c46ea4668edfcbe33a,somewhat qualified
98cb15c740baae03b1fc6a66168196a3,somewhat qualified
4c82c32fe6f4805521da9383e14d6eec,qualified
b08405f6c33908f2d2fecaa65f03ed00,somewhat qualified
95533bae29132008ee40988496c5c2f5,somewhat qualified
9d4306c6029809e681ba2fd3e3aa8e5c,somewhat qualified
d7ec6bca315a9300e6cf2a2ba7102c67,somewhat qualified
77049e7f4deef064e3efcf697a871ade,very qualified
ce9cdd78bc7550c9c752628fd9d3821b,very qualified
b3a896a1cb9dd282e4506809709978c4,very qualified
a31b1841ba4659ed3694238a0de03398,very qualified
0d31495ebfb9cebd26c23ef45b0f04fd,very qualified
6aa5f2c00865da4a45ccdcc7cc75b4ef,very qualified
770c10c0b6e5c8d2de0587922224aed2,very qualified
8e010fdc8b626cc140e41231da29f5cd,very qualified
5d0f01d88acfd3bd9b05f94db58e578a,very qualified
6a955a7c179a252f60c5ef3ed1ae3e42,very qualified
74d8c15bf6e4c704063